In [1]:
import pandas as pd
import json
import datetime
import dateutil
import numpy as np
import os
import xarray as xr

In [2]:
with open('oceansites_flux_list.json') as platform_stream:
    platform_json = json.load(platform_stream)

datasets = platform_json['config']['datasets']
for dataset in datasets:
    if 'locations' in dataset:
        locations_url = dataset['locations']
    if 'url' in dataset:
        url = dataset['url']

    dataset['id'] = url[url.rindex('/') + 1:]
    dataset_info_url = url.replace("tabledap", "info")
    dataset_info_url = dataset_info_url + '/index.csv'
    info_for_dataset = pd.read_csv(dataset_info_url)
    # Add the info for each data set into the dictionary for use when the menu choice changes
    title = list(
        info_for_dataset.loc[(info_for_dataset['Row Type'] == 'attribute') &
                             (info_for_dataset['Variable Name'] == 'NC_GLOBAL') &
                             (info_for_dataset['Attribute Name'] == 'title')]['Value'].unique())[0]
    variables = list(
        info_for_dataset.loc[(info_for_dataset['Row Type'] == 'attribute') &
                             (info_for_dataset['Variable Name'] != 'NC_GLOBAL')]['Variable Name'].unique())
    long_df_all = info_for_dataset.loc[
        (info_for_dataset['Row Type'] == 'attribute') & (info_for_dataset['Attribute Name'] == 'long_name')]
    long_df = long_df_all.drop_duplicates(subset=['Variable Name'])
    long_names = dict(zip(long_df['Variable Name'], long_df['Value'].str.capitalize()))
    unit_df = info_for_dataset.loc[
        (info_for_dataset['Row Type'] == 'attribute') & (info_for_dataset['Attribute Name'] == 'units')]
    units = dict(zip(unit_df['Variable Name'], unit_df['Value']))
    dataset['title'] = title
    if 'site_code' in variables:
        variables.remove('site_code')
    if 'HEIGHT' in variables:
        variables.remove('HEIGHT')
    if 'HEIGHTZS' in variables:
        variables.remove('HEIGHTZS')
    if 'wmo_platform_code' not in variables:
        variables.append('wmo_platform_code')
    # Treat separately below
    if 'time' in variables:
        variables.remove('time')
    if 'latitude' in variables:
        variables.remove('latitude')
    if 'longitude' in variables:
        variables.remove('longitude')
    dataset['variables'] = variables
    dataset['long_names'] = long_names
    dataset['units'] = units
    chk_start_date = info_for_dataset.loc[(info_for_dataset['Row Type'] == 'attribute') & (
            info_for_dataset['Attribute Name'] == 'time_coverage_start') & (
                                                  info_for_dataset['Variable Name'] == 'NC_GLOBAL')]['Value'].to_list()[
        0]

    chk_end_date = info_for_dataset.loc[(info_for_dataset['Row Type'] == 'attribute') & (
            info_for_dataset['Attribute Name'] == 'time_coverage_end') & (
                                                info_for_dataset['Variable Name'] == 'NC_GLOBAL')]['Value'].to_list()[0]

    start_date_datetime = dateutil.parser.isoparse(chk_start_date)
    end_date_datetime = dateutil.parser.isoparse(chk_end_date)

    start_date = start_date_datetime.date().strftime('%Y-%m-%d')
    end_date = end_date_datetime.date().strftime('%Y-%m-%d')

    start_date_seconds = start_date_datetime.timestamp()
    end_date_seconds = end_date_datetime.timestamp()

    dataset['start_date'] = start_date
    dataset['end_date'] = end_date
    mdf = pd.read_csv(locations_url, skiprows=[1],
                      dtype={'wmo_platform_code': str, 'latitude': np.float64, 'longitude': np.float64})

    if mdf.wmo_platform_code.nunique() <= 1:
        adf = mdf.mean(axis=0, numeric_only=True)
        adf['wmo_platform_code'] = mdf['wmo_platform_code'].iloc[0]
        mdf = pd.DataFrame(columns=['latitude', 'longitude', 'wmo_platform_code'], index=[0], )
        mdf['latitude'] = adf.loc['latitude']
        mdf['longitude'] = adf.loc['longitude']
        mdf['wmo_platform_code'] = adf.loc['wmo_platform_code']
    dataset['platforms'] = mdf['wmo_platform_code'].to_list()

In [3]:
dataset = platform_json['config']['datasets'][0]

In [4]:
durl = dataset['url']
did = durl[durl.rindex('/')+1:-1]
all_count_url = durl + '.csv?' + ','.join(dataset['variables'])+',time' + '&orderByCount(\"wmo_platform_code,time/1month\")'

In [5]:
df = pd.read_csv(all_count_url, skiprows=[1])

In [31]:
pd.set_option("max_rows", None)
pd.set_option("max_columns", None)

df2 = df.set_index('wmo_platform_code')
df3 = df2.eq(0).reset_index()
df4 = df3.groupby('wmo_platform_code').sum()
df4

,QLAT,QSEN,QRAIN,SWNET,WZS,TAU,TAUX,TAUY,TDIR,RAIN,EVAP,EMP,FLUX_QC,FLUX_DM,time
wmo_platform_code,,,,,,,,,,,,,,,
13001,75,75,83,75,75,75,75,75,75,29,75,83,75,75,0
13002,18,18,66,18,18,18,18,18,18,59,18,66,18,18,0
13008,163,163,167,163,163,163,182,182,182,52,163,167,163,163,0
13009,154,154,164,154,154,154,154,154,154,62,154,164,154,154,0
13010,122,122,123,122,122,122,122,122,122,65,122,123,122,122,0
13011,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14040,100,100,104,100,100,100,100,100,100,68,100,104,100,100,0
14041,43,43,43,43,43,43,43,43,43,43,43,43,43,43,0
14042,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
